<a href="https://colab.research.google.com/github/DmitryKutsev/DeepHW/blob/master/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

--2020-10-07 16:25:38--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  11.1MB/s    in 64s     

2020-10-07 16:26:43 (10.2 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]



In [2]:
import math
import numpy as np

from tqdm import tqdm

import torch

import zipfile

import seaborn as sns


In [12]:
!wget https://raw.githubusercontent.com/BobaZooba/HSE-Deep-Learning-in-NLP-Course/master/Week%203/data.py

--2020-10-07 17:03:10--  https://raw.githubusercontent.com/BobaZooba/HSE-Deep-Learning-in-NLP-Course/master/Week%203/data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10563 (10K) [text/plain]
Saving to: ‘data.py’

data.py             100%[===================>]  10.32K  --.-KB/s    in 0s      

2020-10-07 17:03:10 (86.0 MB/s) - ‘data.py’ saved [10563/10563]



In [22]:
!mv data.py mydata.py

In [23]:
from mydata import Downloader, Parser

In [24]:
data_path = './data/'

In [25]:
downloader = Downloader(data_path=data_path)

In [26]:
downloader.run()

multiple: 100%|██████████| 17/17 [01:57<00:00,  6.93s/it]


In [27]:
parser = Parser(data_path=data_path)

In [28]:
unlabeled, train, valid = parser.run()

Reading: 100%|██████████| 38/38 [02:33<00:00,  4.03s/it]


In [31]:
unlabeled

,question,response
0,will this fit on ruger 22-45 it has the thread...,"yes, that's how it attaches."
1,not stated anywhere on here nor in any reviews...,"blade thickness: 4.5 mm (0.18""), tapered. http..."
2,"is there a 2"" square hole in the back that cou...",there is a hole. but it is the inside of the 2...
3,"is this a spray or a powder? if powder, is it...","it is a powder and it's a canister, about the ..."
4,celljoy or kaleidio?,celljoy
...,...,...
137275,i did not get any batteries,states it comes with batteries but when i orde...
137276,does this product have corn syrup in it?,not that i know of. when the study sent sample...
137277,does the brother 1230 ribbon fit the brother g...,when i bought my typewriter brother had no ri...
137278,what is the print yield on these cartridges?,"black up to 1,600 standard pages, color up to ..."


In [32]:
train

,question,response,category
0,would this watch be good for someone in their ...,"its fairly large, as far as steps, i drive a o...",sports and outdoors
1,does this require a spanner to install or does...,these pedals have a 6mm insert on the spindle ...,sports and outdoors
2,if nuts are always in the washer with the laun...,good question. all i can tell you is there is ...,beauty
3,does carrier come with infant insert attached ...,"it doesn't have an infant insert, but it has l...",baby
4,does it fit with the spygen neo hybrid case on?,"can't say for sure about the neo hybrid case, ...",cell phones and accessories
...,...,...,...
249995,"what is the length, width and height with the ...","the height is 21 3/4 "" the length is 14"" and w...",pet supplies
249996,does this (cedar ridge rimrock) model come wit...,yes it does,sports and outdoors
249997,do bulbs come with this lamp?,"yes bulbs come with it,",beauty
249998,will this fit a 20575r15 tire. if not do you h...,i have 2001 chevy tracker and it fits find,automotive


In [33]:
valid

,question,response,category
0,does the back support adjust as in going forwa...,no it does not adjust. it has a pin to pull ou...,sports and outdoors
1,can you add weights to increase resistance? i ...,no you cannot....,sports and outdoors
2,are these galvanized?,"i'm not sure, i live in oregon we have rain, s...",sports and outdoors
3,does it work for a razor?,"yes, that what i have......",cell phones and accessories
4,what size is the frame?,the bike comes in multiple frame sizes. ask th...,sports and outdoors
...,...,...,...
49995,does this come with the clip for the phone?,"yes, it does come with a clip.",cell phones and accessories
49996,are there built in adjusting screws for the ai...,you're damn right there are,automotive
49997,can they be worn in the shower & then take you...,no it does have a fabric material to it so the...,beauty
49998,why will this company not ship to texas?! very...,"i am sorry, but that is a glitch on amazon tha...",grocery and gourmet food


In [29]:
# проверим, что в трейне и валидации одинаковые категории
set(train.category.unique().tolist()) == set(valid.category.unique().tolist())

True

In [30]:
train.category.unique()

array(['sports and outdoors', 'beauty', 'baby',
       'cell phones and accessories', 'automotive', 'office products',
       'pet supplies', 'grocery and gourmet food'], dtype=object)

In [34]:
unique_categories = set(train.category.unique().tolist() + valid.category.unique().tolist())

In [35]:
category2index = {category: index for index, category in enumerate(unique_categories)}

In [36]:
category2index

{'automotive': 6,
 'baby': 7,
 'beauty': 5,
 'cell phones and accessories': 4,
 'grocery and gourmet food': 0,
 'office products': 2,
 'pet supplies': 1,
 'sports and outdoors': 3}

In [37]:
train['target'] = train.category.map(category2index)
valid['target'] = valid.category.map(category2index)

In [39]:
from torch.utils.data import Dataset, DataLoader

In [40]:
# игрушечный датасет
# 121535 примера, 4 фичи, 3 класса
some_data_x = np.random.rand(121535, 4)
some_data_y = np.random.randint(3, size=(121535,))

In [41]:

# просто рандомные цифры
some_data_x[:10]

array([[0.68414403, 0.83222526, 0.9783629 , 0.56801603],
       [0.31279536, 0.13508384, 0.22428122, 0.63455277],
       [0.81433872, 0.01006118, 0.9533542 , 0.66111576],
       [0.14798642, 0.43873834, 0.37409874, 0.06996924],
       [0.9768016 , 0.73345217, 0.69669065, 0.69219786],
       [0.8901985 , 0.42117255, 0.03653708, 0.79091627],
       [0.46302592, 0.67427983, 0.81917518, 0.55045393],
       [0.63085536, 0.86645414, 0.95930549, 0.09467927],
       [0.64216652, 0.16270902, 0.89968019, 0.41679377],
       [0.22823859, 0.04596975, 0.14072624, 0.43849377]])

In [42]:
some_data_y

array([2, 0, 0, ..., 2, 2, 2])

In [43]:
batch_size = 16

for i_batch in range(math.ceil(some_data_x.shape[0] / batch_size)):
    
    x_batch = some_data_x[i_batch * batch_size:(i_batch + 1) * batch_size]
    y_batch = some_data_y[i_batch * batch_size:(i_batch + 1) * batch_size]
    
    x_batch = torch.tensor(x_batch)
    y_batch = torch.tensor(y_batch)
    
    break

In [44]:
x_batch

tensor([[0.6841, 0.8322, 0.9784, 0.5680],
        [0.3128, 0.1351, 0.2243, 0.6346],
        [0.8143, 0.0101, 0.9534, 0.6611],
        [0.1480, 0.4387, 0.3741, 0.0700],
        [0.9768, 0.7335, 0.6967, 0.6922],
        [0.8902, 0.4212, 0.0365, 0.7909],
        [0.4630, 0.6743, 0.8192, 0.5505],
        [0.6309, 0.8665, 0.9593, 0.0947],
        [0.6422, 0.1627, 0.8997, 0.4168],
        [0.2282, 0.0460, 0.1407, 0.4385],
        [0.0156, 0.9409, 0.3726, 0.4079],
        [0.5552, 0.9908, 0.1841, 0.3710],
        [0.4304, 0.7264, 0.4705, 0.4469],
        [0.3289, 0.0797, 0.6491, 0.8888],
        [0.6395, 0.1386, 0.6293, 0.8807],
        [0.3954, 0.6375, 0.2258, 0.0162]], dtype=torch.float64)

In [45]:
x_batch.shape, y_batch.shape

(torch.Size([16, 4]), torch.Size([16]))